In [1]:
import numpy as np
from sympy import *
from sympy.matrices import Matrix

First we create the symbols to be used in our Denavit-Hardenberg Calculations

In [58]:
# Create symbols
q1, q2, q3, q4, q5, q6, q7 = symbols('q1:8') # joint angle theta_i
d1, d2, d3, d4, d5, d6, d7 = symbols('d1:8') # link offset
a0, a1, a2, a3, a4, a5, a6 = symbols('a0:7') # link length
alpha0, alpha1, alpha2, alpha3, alpha4, alpha5, alpha6 = symbols('alpha0:7') # twist angle

Then create DH parameters

In [60]:
### KUKA KR210 ###
# DH parameters
s = {alpha0:     0,  a0:      0, d1:  0.75,  
     alpha1: -pi/2,  a1:   0.35, d2:     0, q2: q2-pi/2, 
     alpha2:     0,  a2:   1.25, d3:     0,       
     alpha3: -pi/2,  a3: -0.054, d4:  1.50,
     alpha4:  pi/2,  a4:      0, d5:     0,
     alpha5: -pi/2,  a5:      0, d6:     0,
     alpha6:     0,  a6:      0, d7: 0.303, q7: 0}

Next we define the transformation matrices from the base frame to the gripper frame.

In [38]:
# Define Modified DH Transformation matrix
def build_transformation_matrix( theta, alpha, a, d ):
    T = Matrix([[            cos(theta),           -sin(theta),           0,             a ],
                [ sin(theta)*cos(alpha), cos(theta)*cos(alpha), -sin(alpha), -sin(alpha)*d ],
                [ sin(theta)*sin(alpha), cos(theta)*sin(alpha),  cos(alpha),  cos(alpha)*d ],
                [                     0,                     0,           0,             1 ]]) 
    T = T.subs(s)
       
    return(T)

# Modified DH params
T0_1 = build_transformation_matrix( q1, alpha0, a0, d1 )
T1_2 = build_transformation_matrix( q2, alpha1, a1, d2 )
T2_3 = build_transformation_matrix( q3, alpha2, a2, d3 )
T3_4 = build_transformation_matrix( q4, alpha3, a3, d4 )
T4_5 = build_transformation_matrix( q5, alpha4, a4, d5 )
T5_6 = build_transformation_matrix( q6, alpha5, a5, d6 )
T6_G = build_transformation_matrix( q7, alpha6, a6, d7 )

print("T0_1", T0_1)
print("T1_2", T1_2)
print("T2_3", T2_3)
print("T3_4", T3_4)
print("T4_5", T4_5)
print("T5_6", T5_6)
print("T6_G", T6_G)

T0_1 Matrix([[cos(q1), -sin(q1), 0, 0], [sin(q1), cos(q1), 0, 0], [0, 0, 1, 0.750000000000000], [0, 0, 0, 1]])
T1_2 Matrix([[sin(q2), cos(q2), 0, 0.350000000000000], [0, 0, 1, 0], [cos(q2), -sin(q2), 0, 0], [0, 0, 0, 1]])
T2_3 Matrix([[cos(q3), -sin(q3), 0, 1.25000000000000], [sin(q3), cos(q3), 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])
T3_4 Matrix([[cos(q4), -sin(q4), 0, -0.0540000000000000], [0, 0, 1, 1.50000000000000], [-sin(q4), -cos(q4), 0, 0], [0, 0, 0, 1]])
T4_5 Matrix([[cos(q5), -sin(q5), 0, 0], [0, 0, -1, 0], [sin(q5), cos(q5), 0, 0], [0, 0, 0, 1]])
T5_6 Matrix([[cos(q6), -sin(q6), 0, 0], [0, 0, 1, 0], [-sin(q6), -cos(q6), 0, 0], [0, 0, 0, 1]])
T6_G Matrix([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0.303000000000000], [0, 0, 0, 1]])


Because of the differences between the Denavit-Hardenberg convention and the definition in the URDF-File, we must apply a correctional transform to the last frame to calculate the position in the world-frame. Therefore we first apply a body-fixed rotation around z, then a body-fixed rotation around y.

R_corr is the composition of those two rotations.

In [9]:
#Correct the Gripper
R_z =  Matrix([[     cos(np.pi), -sin(np.pi),             0, 0],
               [     sin(np.pi),  cos(np.pi),             0, 0],
               [              0,           0,             1, 0],
               [              0,           0,             0, 1]])

R_y =  Matrix([[  cos(-np.pi/2),           0, sin(-np.pi/2), 0],
               [              0,           1,             0, 0],
               [ -sin(-np.pi/2),           0, cos(-np.pi/2), 0],
               [              0,           0,             0, 1]])

R_corr = simplify(R_z * R_y)

Now we create individual transformation matrices from base_link to the current link.

In [57]:
# Create individual transformation matrices
T0_2 = simplify(T0_1 * T1_2) # base_link to link_2
T0_3 = simplify(T0_2 * T2_3) # base_link to link_3
T0_4 = simplify(T0_3 * T3_4) # base_link to link_4
T0_5 = simplify(T0_4 * T4_5) # base_link to link_5
T0_6 = simplify(T0_5 * T5_6) # base_link to link_6
T0_G = simplify(T0_6 * T6_G) # base_link to gripper_link
T0_G_corr = simplify(T0_G * R_corr) # Total HT between base_link and gripper_link with orientation correction applied



We extract the end-effector position and orient from the request in ROS. In this notebook, those values are given from the test-dataset. From the given roll, pitch and yaw angles we can calculate a transformation matrix Rrpy which describes the current position and orientation.

Rrpy is a Matrix calculated from a rotation Rx \* Ry \* Rz

In [121]:
# Build a rotation matrix from the Roll, Pitch and Yaw angles
def get_wrist_rot_matrix(roll, pitch, yaw):
        
    R_z =  Matrix([[     cos(yaw), -sin(yaw),             0],
                   [     sin(yaw),  cos(yaw),             0],
                   [              0,           0,             1]])

    R_y =  Matrix([[  cos(pitch),           0, sin(pitch)],
                   [              0,           1,             0],
                   [ -sin(pitch),           0, cos(pitch)]])
    
    R_x = Matrix([[ 1,              0,        0],
                  [ 0,        cos(roll), -sin(roll)],
                  [ 0,        sin(roll),  cos(roll)]])
     
    Rrpy_2 = simplify ( R_z * R_y * R_x )
    print("Rrpy calculated by rotation Rz * Ry * Rz:", Rrpy_2)
    
    
    # Rrpy = Rz * Ry * Rx
    Rrpy = Matrix([[    cos(yaw)*cos(pitch),   cos(yaw)*sin(pitch)*sin(roll) - sin(yaw)*cos(roll),    cos(yaw)*sin(pitch)*cos(roll) + sin(yaw)*sin(roll)],
                   [    sin(yaw)*cos(pitch),   sin(yaw)*sin(pitch)*sin(roll) + cos(yaw)*cos(roll),    sin(yaw)*sin(pitch)*cos(roll) - cos(yaw)*sin(roll)],
                   [            -sin(pitch),             cos(pitch)*sin(roll),                                       cos(pitch)*cos(roll)               ]])
    print("My old Rrpy: ", Rrpy)
    return Rrpy

Rrpy = get_wrist_rot_matrix(q1, q2, q3)

Rrpy calculated by rotation Rz * Ry * Rz: Matrix([
[cos(q2)*cos(q3), sin(q1)*sin(q2)*cos(q3) - sin(q3)*cos(q1),  sin(q1)*sin(q3) + sin(q2)*cos(q1)*cos(q3)],
[sin(q3)*cos(q2), sin(q1)*sin(q2)*sin(q3) + cos(q1)*cos(q3), -sin(q1)*cos(q3) + sin(q2)*sin(q3)*cos(q1)],
[       -sin(q2),                           sin(q1)*cos(q2),                            cos(q1)*cos(q2)]])
My old Rrpy:  Matrix([[cos(q2)*cos(q3), sin(q1)*sin(q2)*cos(q3) - sin(q3)*cos(q1), sin(q1)*sin(q3) + sin(q2)*cos(q1)*cos(q3)], [sin(q3)*cos(q2), sin(q1)*sin(q2)*sin(q3) + cos(q1)*cos(q3), -sin(q1)*cos(q3) + sin(q2)*sin(q3)*cos(q1)], [-sin(q2), sin(q1)*cos(q2), cos(q1)*cos(q2)]])


With Rrpy we can find the location of the wrist center (WC) relative to the base frame. 

l (column 1 of Rrpy) is the vector along gripper_link x-axis.

The end_effector_length is the distance between joint 5 and the tip of the gripper fingers. 

Finally we extend Rrpy with the given positions of the gripper to create a 4x4 transformation matrix we can later use in calculations.

In [122]:
def get_wrist_pos(px, py, pz, Rrpy):
           
    #end_effector_length = 0.453
    end_effector_length = 0.303
    
    
    lx = Rrpy[ 0, 0 ]
    ly = Rrpy[ 1, 0 ]
    lz = Rrpy[ 2, 0 ]

    d6 = 0

    # Calculate Wrist Center
    wx = px - ( end_effector_length + d6 ) * lx
    wy = py - ( end_effector_length + d6 ) * ly
    wz = pz - ( end_effector_length + d6 ) * lz
    
    # Modify Rrpy to take in transformation from gripper
    Rrpy = Rrpy.row_join(Matrix([[px],[py],[pz]]))
    Rrpy = Rrpy.col_join(Matrix([[0,0,0,1]]))

    return wx, wy, wz, Rrpy


Next we find joint variables, theta1, theta2 and theta3.

For Theta 1 we can just take the x and y coordinates we calculated for the wrist center and derive theta1 from tha atan2-function.
![Deriving Theta 1](writeup_images/theta1.png)


Theta 2 is a bit trickier. First, we calculate the wrist center coordinates to match the origin of Frame O_1. Therefor we substract a1 as a horizontal offset from x1 and d1 as a vertical offset from z1. With those new coordinates, we can calculate the distance from Joint2 to Joint5: L25. With the fixed length a3 and d4 we can calculate the distance from Joint3 to Joint5: L35.

Now width the arm-length a2, L25 and L35 we have our triangle and are ready to perform trigonometric calculations. First we calculate the angle *beta2* between a2 and L25 with atan2. For *beta1* we use the cosine-law. As theta2 has a minus 90° Offset, we now have to subtract those two angles from 90°.
![Deriving Theta 2](writeup_images/theta2.png)

For theta3 we can use the already calculated L35 length along with a3 and d4 to calculate a fixed offset angle *beta4*. Additionally, we use the cosine law again to calculate the angle between a2 and L35. To derive theta3 we have to subtract beta3 and beta 4 from the 90° between a2 and d4.
![Deriving Theta 3](writeup_images/theta3.png)

In [131]:
# Calculate joint angles using Geometric IK method
def get_theta_123 (wx, wy, wz):
    
    theta1 = atan2(wy, wx)
    a_1 = 0.35
    a_2 = 1.25
    d_1 = 0.75
    d_4 = 1.50
    a_3 = -0.054
    
  
    r   = sqrt( wx**2 + wy**2) -a_1
    z_c = wz - d_1 # Subtract d1 as vertical offset from robot base
    
    
    L_25 = sqrt(r**2 + z_c**2)
    L_35 = sqrt(a_3**2+d_4**2)
    
    beta_2 = atan2( z_c, r )
    cos_beta_1 = ( ( L_35**2 - L_25**2 - a_2**2 ) / ( - 2 * L_25 * a_2 ) )
    beta_11 = atan2(  sqrt( 1 - cos_beta_1**2 ),  cos_beta_1 ) 
    beta_12 = atan2(  - sqrt( 1 - cos_beta_1**2 ), cos_beta_1 ) 

    theta2 = np.pi/2 - beta_11 - beta_2
    
    beta_4 = atan2(a_3, d_4)
    cos_theta_3 = ( L_25**2 - a_2**2 - L_35**2) / ( -2 * a_2 * L_35 )
    beta_31 = atan2( sqrt( 1 - cos_theta_3**2 ), cos_theta_3 )
    beta_32 = atan2( - sqrt( 1 - cos_theta_3**2 ), cos_theta_3 )
    
    theta3 = np.pi/2 - beta_31 - beta_4
        
    return theta1.evalf(), theta2.evalf(), theta3.evalf()

Once the first three joint variables are known, we calculate R0_3 via application of homogeneous transforms up to the WC. Then we calculate the R3_6_var Matrix, which contains the remaining unknown thetas 4-6.

By multiplying the inverse of R0_3 with Rrpy we can retrieve R3_6_eval, which contains numerical values. This is important for the next step, where we use those two matrices to generate equations to solve for the thetas.

In [65]:
def transform_to_wc(theta1, theta2, theta3, Rrpy):
 
    R0_3 = simplify( T0_1 * T1_2 * T2_3 )
    R0_3 = R0_3.evalf(subs={q1: theta1, q2: theta2, q3: theta3})
    R3_6_eval = R0_3.inv() * Rrpy
    R3_6_var = simplify( T3_4 * T4_5 * T5_6 )
    
    return R3_6_var, R3_6_eval        

To get the composite rotation matrix R3_6 we multiply the precalculated T3_4, T4_5 and T5_6.
![Deriving R3_6](writeup_images/R3_6.png)

In [66]:
R3_6_var, R3_6_eval = transform_to_wc( theta1, theta2, theta3, Rrpy )
print("R3_6:", R3_6_var)

R3_6: Matrix([
[-sin(q4)*sin(q6) + cos(q4)*cos(q5)*cos(q6), -sin(q4)*cos(q6) - sin(q6)*cos(q4)*cos(q5), -sin(q5)*cos(q4), -0.054],
[                           sin(q5)*cos(q6),                           -sin(q5)*sin(q6),          cos(q5),    1.5],
[-sin(q4)*cos(q5)*cos(q6) - sin(q6)*cos(q4),  sin(q4)*sin(q6)*cos(q5) - cos(q4)*cos(q6),  sin(q4)*sin(q5),      0],
[                                         0,                                          0,                0,      1]])


From this matrix we choose r_ij to calculate our angles. Those are the r_ji I choose for my calculations:

In [54]:
print("r23 = ", R3_6_var[1,2])
print("r13 = ", R3_6_var[0,2])
print("r33 = ", R3_6_var[2,2])

print("r22 = ", R3_6_var[1,1])
print("r21 = ", R3_6_var[1,0])

r23 =  cos(q5)
r13 =  -sin(q5)*cos(q4)
r33 =  sin(q4)*sin(q5)
r22 =  -sin(q5)*sin(q6)
r21 =  sin(q5)*cos(q6)


Now we find a set of Euler angles corresponding to the rotation matrix.

We start with Theta 5 as r23 contains an isolated cos(q5). to get sin(q5) we calculate sqrt(r13^2 + r33^2), as they contain sin(q4) and cos(q4). Those combined equate to 1 and only leave sin(q5) remaining.
![Deriving Theta 5](writeup_images/theta5.png)

Now that we have q5 calculated, we can choose r_ij containing only q4 and q5 to calculate theta4.
![Deriving Theta 4](writeup_images/theta4.png)

And we can choose r_ij containing only q6 and q5 to calculate theta6.
![Deriving Theta 6](writeup_images/theta6.png)

In [69]:
def get_theta_456(R3_6_var, R3_6_eval):
    r13 = R3_6_eval[0,2]
    r33 = R3_6_eval[2,2]
    r23 = R3_6_eval[1,2]
    r22 = R3_6_eval[1,1]
    r21 = R3_6_eval[1,0]
        
    sin_q5 = sqrt(r13**2 + r33**2).evalf()
           
    theta5 = atan2( sin_q5, r23 ).evalf()
    
    if( sin_q5 < 0 ):
        theta4 = atan2(-r33,  r13).evalf()
        theta6 = atan2( r22, -r21).evalf()
    else:
        theta4 = atan2( r33,  -r13).evalf()
        theta6 = atan2( -r22, r21).evalf()
           
    return theta4, theta5, theta6

Now that we have all our values calculated, lets see what we can derive from the test-data.
This is some test-input data for verifying the Inverse Kinematics using Forward Kinematics.

In [125]:
test_set_1 = { "xpos" : 2.3029, "ypos" : 0, "zpos" : 1.9466, "roll" : 0, "pitch" : 0, "yaw" : 0 , 
               "theta1_rviz" : 0, "theta2_rviz" : 0, "theta3_rviz" : 0, 
               "theta4_rviz" : 0, "theta5_rviz" : 0, "theta6_rviz" : 0,
               "wcx_rviz" : 1.84990000000000, "wcy_rviz" : 0, "wcz_rviz" : 1.94660000000000,}
test_set_2 = { "xpos" : 2.16135, "ypos" : -1.42635, "zpos" : 1.5511, 
               "roll" : 1.6544,  "pitch" : 0.4899, "yaw" : 0.0624, 
               "theta1_rviz" : -0.65, "theta2_rviz" : -0.6500, "theta3_rviz" : -0.65, 
               "theta4_rviz" : 0.95, "theta5_rviz" : 0.79, "theta6_rviz" : 0.49,
               "wcx_rviz" : 1.89451, "wcy_rviz" : -1.44302, "wcz_rviz" : 1.69366,}
test_set_2 = { "xpos" : -0.56754, "ypos" : 0.93663, "zpos" : 3.0038, 
               "roll" : 0.62073, "pitch" : 0.48318, "yaw" : 0.38759, 
               "theta1_rviz" : -0.65, "theta2_rviz" : 0.45, "theta3_rviz" : -0.36, 
               "theta4_rviz" : 0.95, "theta5_rviz" : 0.79, "theta6_rviz" : 0.49,
               "wcx_rviz" : -0.638, "wcy_rviz" : 0.64198, "wcz_rviz" : 2.9988,}

# Choose current set of test data
current_test_set = test_set_2

In [126]:
Rrpy                                = get_wrist_rot_matrix(current_test_set["roll"], current_test_set["pitch"], current_test_set["yaw"])
test_wx,  test_wy,    test_wz, Rrpy = get_wrist_pos(current_test_set["xpos"], current_test_set["ypos"], current_test_set["zpos"], Rrpy)
theta1,   theta2,     theta3        = get_theta_123( test_wx, test_wy, test_wz )
R3_6_var, R3_6_eval                 = transform_to_wc( theta1, theta2, theta3, Rrpy )
theta4,   theta5,     theta6        = get_theta_456( R3_6_var, R3_6_eval )


Rrpy calculated by rotation Rz * Ry * Rz: Matrix([
[ 0.81983628831082, -0.0572727414660277,  0.569726507592637],
[0.334690367557598,   0.855247381121388, -0.395644126645611],
[-0.46459747979611,     0.5150453865146,  0.720331473418404]])
My old Rrpy:  Matrix([[0.819836288310820, -0.0572727414660277, 0.569726507592637], [0.334690367557598, 0.855247381121388, -0.395644126645611], [-0.464597479796110, 0.515045386514600, 0.720331473418404]])


In [127]:
def calculate_FK_pose( self, theta1, theta2, theta3, theta4, theta5, theta6 ):

    EEpos = self.T0_G_corr.evalf( subs={ self.q1: theta1, self.q2: theta2, self.q3: theta3, self.q4: theta4, self.q5: theta5, self.q6: theta6 } )
    Test = Matrix([[1.0],[1.0],[1.0],[1.0]])
    EEpos = simplify( EEpos * Test )
    ex = EEpos[0,0]
    ey = EEpos[1,0]
    ez = EEpos[2,0]
    return ex, ey, ez

def calculate_J5_pose( self, theta1, theta2, theta3, theta4, theta5, theta6 ):

    J5pos = simplify( self.T0_5 * self.R_corr )
    new_J5 = J5pos.evalf( subs={ self.q1: theta1, self.q2: theta2, self.q3: theta3, self.q4: theta4, self.q5: theta5, self.q6: theta6 } )
    Test = Matrix([[1.0],[1.0],[1.0],[1.0]])
    new_J5 = simplify( new_J5 * Test )
    jx = new_J5[0,0]
    jy = new_J5[1,0]
    jz = new_J5[2,0]
    return jx, jy, jz



calculate_FK_pose( -0.6509, 0.2529, -0.3261, -1.4192, 0.9081, -0.6427 )

TypeError: calculate_FK_pose() missing 1 required positional argument: 'theta6'

And now for we compare the calculated values against expected values from the sim

In [128]:
print("---------------- input data ----------------")
print("px, py, pz:   ", current_test_set["xpos"], current_test_set["ypos"], current_test_set["zpos"])
print("r , p , y :   ", current_test_set["roll"], current_test_set["pitch"], current_test_set["yaw"],"\n")
print("--------------- wrist center ---------------")
print("WC_x: ", test_wx, "expected: ", current_test_set["wcx_rviz"])
print("WC_y: ", test_wy, "expected: ", current_test_set["wcy_rviz"])
print("WC_z: ", test_wz, "expected: ", current_test_set["wcz_rviz"],"\n")
print("--------------- joint angles ---------------")
print("theta 1:", theta1, "expected: ", current_test_set["theta1_rviz"])
print("theta 2:", theta2, "expected: ", current_test_set["theta2_rviz"])
print("theta 3:", theta3, "expected: ", current_test_set["theta3_rviz"])
print("theta 4:", theta4, "expected: ", current_test_set["theta4_rviz"])
print("theta 5:", theta5, "expected: ", current_test_set["theta5_rviz"])
print("theta 6:", theta6, "expected: ", current_test_set["theta6_rviz"],"\n")
print("--------------- final matrices ---------------")
theta_values = { q1: theta1, q2: theta2, q3: theta3, q4: theta4, q5: theta5, q6: theta6 }
print("T0_G_corr: ", T0_G_corr.evalf(subs=theta_values))
print("Rrpy: ", Rrpy.evalf())

---------------- input data ----------------
px, py, pz:    -0.56754 0.93663 3.0038
r , p , y :    0.62073 0.48318 0.38759 

--------------- wrist center ---------------
WC_x:  -0.815950395358179 expected:  -0.638
WC_y:  0.835218818630048 expected:  0.64198
WC_z:  3.14457303637822 expected:  2.9988 

--------------- joint angles ---------------
theta 1: 2.34452545680045 expected:  -0.65
theta 2: -0.730973161577067 expected:  0.45
theta 3: -1.02650253231907 expected:  -0.36
theta 4: -2.16144899394904 expected:  0.95
theta 5: 0.0428143668228980 expected:  0.79
theta 6: -0.436237779464465 expected:  0.49 

--------------- final matrices ---------------
T0_G_corr:  Matrix([
[ 0.171374330942821,  0.254024895969983, -0.951894001935793, 0.547958761407931],
[-0.124549223196753,  0.964022052079091,  0.234838187069958, -0.54548440831795],
[ 0.977301555148147, 0.0783124213178125,  0.196847237654876,  3.46073988187786],
[                 0,                  0,                  0,               1.0